In [26]:
#from elasticsearch import Elasticsearch
import os
import json
from jsmin import jsmin
import pyelasticsearch
from pyelasticsearch import ElasticSearch #see if this is better than 'official API' - the docs look better to me
from pyelasticsearch import bulk_chunks


In [2]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [3]:
%load_ext watermark

In [47]:
%watermark -a "Samantha Zeitlin" -d -u -p pyelasticsearch,jsmin,python 

Samantha Zeitlin 
Last updated: 04/11/2015 

pyelasticsearch 1.2.1
jsmin 2.1.1
python 2.7.9


In [27]:
try:
    es = ElasticSearch('http://localhost:9200')
except:
    ConnectionError
    print "I can't connect!"

In [28]:
#start fresh to see if mapping first is better than mapping after
es.delete_all_indexes() #the deletion worked, but then it gave an error b/c there was nothing to delete. 
#print ElasticSearch.send_request.__doc__
try:
    es.send_request("GET", ["_status"],{},{}) 
except ElasticHttpError as vague:
    print "look at vague error message"

ElasticHttpError: Non-OK response returned (400): u'ActionRequestValidationException[Validation Failed: 1: index / indices is missing;]'

In [75]:
!pwd

/Users/szeitlin/mystuff/projects/nbindex/nbindex


In [10]:
#may need to initialize an index first, then bulk insert, then map?
print ElasticSearch.create_index.__doc__


        Create an index with optional settings.

        :arg index: The name of the index to create
        :arg settings: A dictionary of settings

        If the index already exists, raise
        :class:`~pyelasticsearch.exceptions.IndexAlreadyExistsError`.

        See `ES's create-index API`_ for more detail.

        .. _`ES's create-index API`:
            http://www.elastic.co/guide/en/elasticsearch/reference/current/indices-create-index.html
        


In [29]:
es.create_index('test3')

IndexAlreadyExistsError: Non-OK response returned (400): u'IndexAlreadyExistsException[[test3] already exists]'

In [38]:
#put a few target files in folder and iterate through that)

path = '/Users/szeitlin/mystuff/projects/nbindex/nbindex/testdir'
# newline = '\n'

def onefile(newline):
    """
    Test generator on one notebook file. 
    """
    with open('testdir/compare_2013_2014.ipynb', 'r') as one:
        yield jsmin(one.read()) + newline
        
for chunk in bulk_chunks(onefile(newline)):
    try:
        es.bulk(chunk, index='test3')
    except pyelasticsearch.exceptions.BulkError as bulk_failure:
        result = "check bulk_failure"
    except pyelasticsearch.exceptions.ElasticHttpError as vague:
        result = "not sure"

# def notebooks(path, newline):
#     for eachfile in os.listdir(path):
#         fullname = os.path.join(path,eachfile)
#         with open(fullname, 'r') as each:
#             yield jsmin(each.read()) + newline
 
# for chunk in bulk_chunks(notebooks(path, newline)):
#     print chunk
#     try:
#         es.bulk(chunk, doc_type='notebook', index='test3')
#     except pyelasticsearch.exceptions.BulkError as bulk_failure:
#         result = "check bulk_failure"
#     except pyelasticsearch.exceptions.ElasticHttpError as vague:
#         result = "not sure"
        
        

In [39]:
result

'not sure'

In [40]:
vague

pyelasticsearch.exceptions.ElasticHttpError(400,
                                            u'ActionRequestValidationException[Validation Failed: 1: no requests added;]')

In [6]:
#looks like it wants you to create an index first and put_mapping second
#print es.put_mapping.__doc__

with open("mapping_minus_curl.json", 'r') as map:
     minimap = jsmin(map.read())
     es.put_mapping(index='test3', doc_type='notebook', mapping=minimap)

ElasticHttpNotFoundError: Non-OK response returned (404): u'IndexMissingException[[test3] missing]'

In [64]:
#replace this with bulk_chunks instead 
#es.bulk([es.index_op(minified), es.index_op(mini2)], doc_type='notebook', index='test2')

In [66]:
!ls

Dockerfile                                 another_query.txt                          first_try_mapping.json                     minified_testfile.json                     testcreate.json
LICENSE                                    draft_bulk_import_multiple_notebooks.ipynb import_statements_query.txt                pyminified_testfile.json                   testnb.json
README.md                                  elasticsearch.yml                          match_query.txt                            test2.json                                 testprefix.json
action.json                                elk_test                                   minified.json                              testbulk.json                              testquery.txt


In [48]:
with open('nocurl_query.txt', 'r') as query:
    q=jsmin(query.read())
    #print q
    #says it wants a \n character? 
    newline = '\n'
    try:
        r = es.search(str(q + newline), index='test2')
        print r
    except: 
        e = pyelasticsearch.ElasticHttpError
        print e

<class 'pyelasticsearch.exceptions.ElasticHttpError'>


In [21]:
%%bash
curl -s -XPOST localhost:9200/_bulk --data-binary @test2.json


{"took":251,"errors":false,"items":[{"index":{"_index":"test","_type":"type1","_id":"2","_version":1,"status":201}}]}

In [ ]:
#looks like kibana barely works at all with only 1 document, suddenly with 2 it starts showing more functionality. 